In [91]:
using LowLevelFEM
using LinearAlgebra

In [92]:
function square_mesh(; x0=0.0, y0=0.0, lx=1.0, ly=1.0, n=10, dx=lx / n, dy=ly / n, order=1)

    gmsh.option.setNumber("General.Verbosity", 0)
    gmsh.model.add("square")
    # --------------------------------------------------
    # Geometry
    # --------------------------------------------------
    # Box: origin (0,0,0), size l x l x l
    box = gmsh.model.occ.addRectangle(x0, y0, 0.0, lx, ly)

    gmsh.model.occ.synchronize()
    #return

    # --------------------------------------------------
    # Physical groups
    # --------------------------------------------------
    gmsh.model.addPhysicalGroup(1, [1], -1, "bottom")
    gmsh.model.addPhysicalGroup(1, [2], -1, "right")
    gmsh.model.addPhysicalGroup(1, [3], -1, "top")
    gmsh.model.addPhysicalGroup(1, [4], -1, "left")

    gmsh.model.addPhysicalGroup(2, [1], -1, "surface")

    gmsh.model.addPhysicalGroup(0, [1], -1, "leftbottom")
    gmsh.model.addPhysicalGroup(0, [2], -1, "rightbottom")
    gmsh.model.addPhysicalGroup(0, [3], -1, "righttop")
    gmsh.model.addPhysicalGroup(0, [4], -1, "lefttop")

    # --------------------------------------------------
    # Mesh settings (structured hex mesh)
    # --------------------------------------------------
    #gmsh.option.setNumber("Mesh.Algorithm3D", 1)      # Delaunay for recombination
    gmsh.option.setNumber("Mesh.RecombineAll", 1)
    #gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 1)

    #gmsh.option.setNumber("Mesh.SubdivisionAlgorithm", 1)

    d = min(dx, dy)

    gmsh.model.mesh.setTransfiniteCurve(1, ceil(lx / d) + 1)
    gmsh.model.mesh.setTransfiniteCurve(3, ceil(lx / d) + 1)
    gmsh.model.mesh.setTransfiniteCurve(2, ceil(ly / d) + 1)
    gmsh.model.mesh.setTransfiniteCurve(4, ceil(ly / d) + 1)

    gmsh.model.mesh.setTransfiniteSurface(1)

    gmsh.option.setNumber("Mesh.ElementOrder", order)

    # Characteristic length control
    #lc = min(dx, dy)
    #gmsh.option.setNumber("Mesh.CharacteristicLengthMin", lc)
    #gmsh.option.setNumber("Mesh.CharacteristicLengthMax", lc)

    # --------------------------------------------------
    # Generate mesh
    # --------------------------------------------------
    gmsh.model.mesh.generate(2)

    # --------------------------------------------------
    # Save
    # --------------------------------------------------
    #gmsh.write("cube.msh")

    #gmsh.finalize()

    return nothing
end


square_mesh (generic function with 1 method)

In [93]:
gmsh.initialize()
square_mesh(order=4)

In [94]:
mat = Material("surface")
prob = Problem([mat], type=:ScalarField, dim=2)

Problem("square", :ScalarField, 2, 1, Material[Material("surface", :Hooke, 200000.0, 0.3, 115384.61538461536, 76923.07692307692, 166666.66666666663, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 1.0e-7, 0.1, 1.0)], 1.0, 1681, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [95]:
ϕ1 = ScalarField(prob, "surface", (x, y, z) -> x^4)
ϕ2 = ScalarField(prob, "surface", (x, y, z) -> x^3 * y)
ϕ3 = ScalarField(prob, "surface", (x, y, z) -> x^2 * y^2)
ϕ4 = ScalarField(prob, "surface", (x, y, z) -> x^1 * y^3)
ϕ5 = ScalarField(prob, "surface", (x, y, z) -> y^4)

elementwise ScalarField
[[0.0; 0.0; … ; 6.249999999999994e-6; 6.249999999999966e-6;;], [9.999999999999991e-5; 9.999999999999991e-5; … ; 0.0005062499999999988; 0.0005062499999999988;;], [0.0015999999999999986; 0.0015999999999999986; … ; 0.003906249999999993; 0.003906249999999993;;], [0.008099999999999993; 0.008099999999999993; … ; 0.015006249999999978; 0.015006249999999978;;], [0.025599999999999977; 0.025599999999999977; … ; 0.04100624999999999; 0.04100624999999999;;], [0.0625; 0.0625; … ; 0.09150624999999996; 0.09150624999999996;;], [0.1296; 0.1296; … ; 0.1785062499999999; 0.1785062499999999;;], [0.24009999999999995; 0.24009999999999995; … ; 0.31640625; 0.31640625;;], [0.4096000000000001; 0.4096000000000001; … ; 0.5220062499999997; 0.5220062500000002;;], [0.6561; 0.6561; … ; 0.8145062499999999; 0.8145062499999999;;]  …  [0.0; 0.0; … ; 6.249999999999994e-6; 6.249999999999966e-6;;], [9.999999999999991e-5; 0.00010000000000000002; … ; 0.0005062500000000003; 0.0005062500000000003;;], [0.001

In [96]:
∫(prob, "right", ∂y(∂y(ϕ1)))

-2.996016124517983e-11

In [97]:
∫(prob, "right", ∂y(∂y(ϕ2)))

-1.4610062288354294e-11

In [98]:
∫(prob, "right", ∂y(∂y(ϕ3)))

1.9999999999904094

In [99]:
∫(prob, "right", ∂y(∂y(ϕ4)))

2.999999999992722

In [100]:
∫(prob, "right", ∂y(∂y(ϕ5)))

3.9999999999941593

In [101]:
top = constrainedDoFs(prob, [BoundaryCondition("top", p=0)]);

In [102]:
norm(DoFs(elementsToNodes(∂y(∂y(ϕ1))))[top, 1])

2.7260391117377398e-11

In [103]:
norm(DoFs(elementsToNodes(∂y(∂y(ϕ2))))[top, 1])

3.16388347074544e-11

In [104]:
norm(DoFs(elementsToNodes(∂y(∂y(ϕ3))))[top, 1])

5.833809325822822

In [105]:
norm(DoFs(elementsToNodes(∂y(∂y(ϕ4))))[top, 1])

22.31927418175065

In [106]:
norm(DoFs(elementsToNodes(∂y(∂y(ϕ5))))[top, 1])

76.83749084919518

In [107]:
gmsh.finalize()